In [1]:
import random

import numpy as np
import torch
import math
import torch.nn as nn
from PIL import Image
from torch.autograd import Variable
from torchvision.transforms import transforms

from commons import inc_convolution, inc_max_pool
from imagenet_classes import class_names
from vgg16 import VGG16

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
batch_size = 1
patch_size = 16
input_size = 224

p = (input_size - patch_size)//2
patch_locations = [(p, p)]*batch_size

loader = transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()])
images = Image.open('./dog_resized.jpg')
images = loader(images)

images = images.unsqueeze(0)
images_batch = images.repeat(batch_size, 1, 1, 1)

image_patch = torch.cuda.FloatTensor(3, patch_size, patch_size).fill_(0)

for i,(x,y) in enumerate(patch_locations):
    images_batch[i, :, x:x+patch_size, y:y+patch_size] = image_patch

y = VGG16().forward(images_batch.cuda())

image_patches = image_patch.unsqueeze(0).repeat(batch_size, 1, 1, 1)
patch_locations = torch.from_numpy(np.array(patch_locations, dtype=np.int32))

inc_model = VGG16(beta=0.5)

inc_model.eval()
inc_model.forward_materialized(images)
x = inc_model.forward_gpu(image_patches, patch_locations, patch_size, patch_size)

(tensor([[ 103,  103]], dtype=torch.int32, device='cuda:0'), 18)
(tensor([[ 102,  102]], dtype=torch.int32, device='cuda:0'), 20)
(tensor([[ 51,  51]], dtype=torch.int32, device='cuda:0'), 11)
(tensor([[ 50,  50]], dtype=torch.int32, device='cuda:0'), 13)
(tensor([[ 49,  49]], dtype=torch.int32, device='cuda:0'), 15)
(tensor([[ 24,  24]], dtype=torch.int32, device='cuda:0'), 8)
(tensor([[ 23,  23]], dtype=torch.int32, device='cuda:0'), 10)
(tensor([[ 22,  22]], dtype=torch.int32, device='cuda:0'), 12)
(tensor([[ 21,  21]], dtype=torch.int32, device='cuda:0'), 14)
(tensor([[ 10,  10]], dtype=torch.int32, device='cuda:0'), 8)
(tensor([[ 9,  9]], dtype=torch.int32, device='cuda:0'), 10)
(tensor([[ 8,  8]], dtype=torch.int32, device='cuda:0'), 12)
(tensor([[ 7,  7]], dtype=torch.int32, device='cuda:0'), 14)
(tensor([[ 4,  4]], dtype=torch.int32, device='cuda:0'), 7)
(tensor([[ 4,  4]], dtype=torch.int32, device='cuda:0'), 7)
(tensor([[ 4,  4]], dtype=torch.int32, device='cuda:0'), 7)
(tens

In [3]:
s = 100
p = 62
#temp = x - y[:,:,p:p+s,p:p+s]

temp = y-x

temp = temp.cpu().data.numpy()
print(np.max(np.abs(temp)))

2.3841858e-07
